# Install the New Worker- BETA!!
### Remember to first set your API KEY and worker name
#### You can serve different models, simply change the name in models_to_load to match the model you want, you can check either https://aqualxx.github.io/stable-ui/workers, in the models tab, or https://tinybots.net/artbot/info/models, just copy paste the name (it's set to Deliberate by default)
#### You can also change max_power and see how high you can go, it's set to 20 by default

In [ ]:
# @title ## Only to obtain the models names from the horde. If you want it, tick the checkbox here and run this cell. It does nothing for the worker, it will simply print a list of all the models { display-mode: "form" }
get_models = False # @param {type:"boolean"}

if (get_models):

  !wget -q -O /content/stable_diffusion.json https://raw.githubusercontent.com/Haidra-Org/AI-Horde-image-model-reference/main/stable_diffusion.json

  with open("/content/stable_diffusion.json", 'r') as file:
      lines = file.readlines()

  delete_name = '        "name": '
  delete_enter = '\n'
  model_list = []

  for i, line in enumerate(lines):
      if delete_name in line:
          #print(f"{line.replace(delete_name, '').replace(delete_enter,'')}")
          #model_list.append = [f"{line.replace(delete_name, '').replace(delete_enter,'')}"]
          model_list.append(line.replace(delete_name, '').replace(delete_enter,''))

  model_list.sort()  # Sort the model_list in alphabetical order

  #print(model_list)
  print(' '.join(model_list))


In [ ]:
# @title # Chronometer (time listener) { display-mode: "form" }
%%javascript
const listenerChannel = new BroadcastChannel('channel');
listenerChannel.onmessage = (msg) => {
  const div = document.createElement('div');
  div.textContent = msg.data;
  document.body.innerHTML = '';
  document.body.appendChild(div);
};

const div = document.createElement('listener');
document.body.appendChild(div);


In [ ]:
# @title # Chronometer (time sender) { display-mode: "form" }
%%javascript
const senderChannel = new BroadcastChannel('channel');

let startTime = new Date().getTime(); // Variable to store the start time

function formatTime(seconds) {
  const hours = Math.floor(seconds / 3600);
  const minutes = Math.floor((seconds % 3600) / 60);
  const remainingSeconds = seconds % 60;
  return `${hours} hours, ${minutes} minutes, ${remainingSeconds} seconds`;
}

function updateTime() {
  const currentTime = new Date().getTime();
  const elapsedSeconds = Math.floor((currentTime - startTime) / 1000);
  const formattedTime = formatTime(elapsedSeconds);
  senderChannel.postMessage(`Time elapsed: ${formattedTime}`);
}

setInterval(updateTime, 1000)
const div = document.createElement('sender');
div.textContent = "Running timer! These 2 cells will run alongside the worker, keeping track of the time elapsed since starting the notebook. Closing this output wil stop the timer (at least, I think)!";
document.body.appendChild(div);


In [ ]:
# @title 0.- This cell will set the variables and rerun the worker if you stopped it, but only if everything else was installed already { display-mode: "code" }

#### For right now, THESE are the only variables that we care about
worker_name = ""
api_key = ""
max_power = 20
safety_on_gpu = True
models_to_load = ["Deliberate"]
allow_lora = True
allow_controlnet = True
nsfw = True
censor_nsfw = False

##############################
##### HIGHLY EXPERIMENTAL - OPTIONAL, disabled by default ####
# Colab exclusive, for now, inject loras and textual inversions through prompt so they can be used in similar fashion to A1111
# for loras: <lora:civitaiID:lora_strength:lora_clip, Anything you want here, to identify this lora>
# for textual inversions: (embedding:civitaiID:ti_strength)
experimental = False
# why? because some front-ends don't support loras/tis, so instead we let the worker handle that part and now EVERYONE can use them,
# no  matter what client they use to access the horde
#### HIGHLY EXPERIMENTAL - OPTIONAL, disabled by default  ####
##############################

####For right now, THESE are the only variables that we care about


queue_size = 1
max_threads = 1
horde_url = "https://aihorde.net"
allow_painting = False
dynamic_models = False
models_to_skip = ["stable_diffusion_inpainting", "stable_diffusion_2.1",  "stable_diffusion_2.0"]
allow_post_processing = False
priority_usernames = []
blacklist = []
censorlist = []
allow_img2img = True
allow_unsafe_ip = True
number_of_dynamic_models = 0
max_models_to_download = 10
forms = ["caption","nsfw","interrogation","post-process"]


current_path = "/content/"
worker_path = current_path + "horde-worker-reGen/"
bridgeData_file = worker_path + "bridgeData.yaml"

import os

#@markdown To reduce the level of verbosity/show reduced logs (Colab uses more and more of your PC RAM as the logs grow longer, but this will help with that)
Short_Logs = True #@param {type:"boolean"}
logs = Short_Logs

if not os.path.exists(bridgeData_file):
    print ("bridgeData.yaml file not found. Proceed to install the worker.")
else:
    print ("bridgeData.yaml file found. Recreating bridgeData.yaml and restarting the worker.")
    create_yaml()
    !cd /content
    !source ../regen/bin/activate;python download_models.py
    !cd /content
    
    if (Short_Logs):
        !source ../regen/bin/activate;python run_worker.py -vv
    else:
        !source ../regen/bin/activate;python run_worker.py


In [ ]:
# @title 1.- Virtual Environment  { display-mode: "form" }

!apt-get update
!apt install python3.10-venv
!python -m venv regen

In [ ]:
# @title 2.- Remove the worker if it exists, then, Clone the regen worker { display-mode: "form" }

!cd /content;rm -r horde-worker-reGen
!cd /content;git clone https://github.com/Haidra-Org/horde-worker-reGen.git

In [ ]:
# @title 3.- Install requirements for cuda 11.8 { display-mode: "form" }

!source regen/bin/activate;pip install -r .\/horde-worker-reGen/requirements.118.txt

In [ ]:
# @title 4.- Work around so hordelib installs @ cuda 11.8 { display-mode: "form" }

%cd /content/

!source regen/bin/activate;pip install -r .\/horde-worker-reGen/requirements.hordelib.txt

!source regen/bin/activate;pip install hordelib --no-deps
#!source regen/bin/activate;pip install hordelib~=2.3.3

In [ ]:
# @title 5.- Create .yaml config file { display-mode: "form" }

import os

%cd $worker_path
print ("Creating bridgeData.yaml file.")

def create_yaml():

    from yaml import load, dump

    def make_yaml_sublist(list_to_convert: list[str]):
        sublist_yaml = dump(list_to_convert)
        sublist_yaml = "\n" + sublist_yaml
        return sublist_yaml



    data = f"""horde_url: "{horde_url}"
api_key: "{api_key}"
priority_usernames: []
max_threads: {max_threads}
queue_size: {queue_size}
safety_on_gpu: {safety_on_gpu}
require_upfront_kudos: false
cycle_process_on_model_change: true
dreamer_name: "{worker_name}"
max_power: {max_power}
nsfw: {nsfw.__str__().lower()}
censor_nsfw: {censor_nsfw}
blacklist: {blacklist}
censorlist: {censorlist}
allow_img2img: {allow_img2img.__str__().lower()}
allow_painting: {allow_painting.__str__().lower()}
allow_unsafe_ip: true
allow_post_processing: {allow_post_processing.__str__().lower()}
allow_controlnet: {allow_controlnet.__str__().lower()}
allow_lora: {allow_lora.__str__().lower()}
max_lora_cache_size: 20
dynamic_models: false
number_of_dynamic_models: 0
max_models_to_download: 10
stats_output_frequency: 30
cache_home: "./"
always_download: true
temp_dir: "./tmp"
disable_terminal_ui: True
vram_to_leave_free: "80%"
ram_to_leave_free: "80%"
disable_disk_cache: false
models_to_load: {make_yaml_sublist(models_to_load)}
models_to_skip: {make_yaml_sublist(models_to_skip)}
suppress_speed_warnings: false
forms:
- "caption"
- "nsfw"
- "interrogation"
- "post-process"
"""

    with open(bridgeData_file, "w") as text_file:
        text_file.write(data)

    print ("bridgeData.yaml file created.")

create_yaml()

In [ ]:
# @title ## EXPERIMENTAL - Lora/TI prompt injection and Hires-Fix Denoise Strength and initial image size control { display-mode: "form" }

# To inject loras and tis through the prompt like auto1111 and other UIs

# To undo the experimental changes and disable the loras/tis injection, uncomment the next 2 lines (delete the #) and run this cell again, then rerun the worker (the last cell)
#!wget -O /content/regen/lib/python3.10/site-packages/hordelib/horde.py https://raw.githubusercontent.com/Haidra-Org/hordelib/v2.2.3/hordelib/horde.py
#experimental = False


# To inject loras and tis through the prompt, we search in horde.py for "search_string" and write "new_code" above it, just because

horde_py_path = '/content/regen/lib/python3.10/site-packages/hordelib/horde.py'
search_string = '        # Negative and positive prompts are merged together'
new_code = r'''
        #
        # To inject Loras and TIs through the prompt
        # Import regular expressions
        import re

        # Get the lora(s) from the prompt
        matches = re.findall(r"<lora:(-?\d+:-?\d+\.\d+:-?\d+\.\d+).*?>", payload.get("prompt"))
        lora_array = []
        # Array containing all loras in the prompt
        for match in matches:
            lora_array.append(match)
        # Remove the lora(s) from the prompt
        payload["prompt"] = re.sub(r"<lora:-?\d+:-?\d+\.\d+:-?\d+\.\d+.*?>", "", payload["prompt"])
        
        # Append the new lora(s) to payload["loras"] 
        for lora_string in lora_array:
            attributes = ["name", "model", "clip"]
            values = lora_string.split(':')
            lora_dict = dict(zip(attributes, values))
            payload["loras"].append(lora_dict)

        # Get the ti(s) from the prompt
        matches = re.findall(r"embedding:(-?\d+:-?\d+\.\d+)", payload.get("prompt"))
        ti_array = []
        # Array containing all loras in the prompt
        for match in matches:
            ti_array.append(match)
        
        # Append the new TI(s) to payload["tis"] 
        for ti_string in ti_array:
            attributes = ["name", "strength"]
            # if inject_ti is necessary, use this next line instead
            #attributes = ["name", "inject_ti", "strength"]
            values = ti_string.split(':')
            ti_dict = dict(zip(attributes, values))
            payload["tis"].append(ti_dict)

        # To inject Loras and TIs through the prompt
        #

        #
        # To change hires_fix_denoising_strength
        import re
        # Get the value from the prompt
        match = re.search(r"<hires_fix:(\d+\.\d+)>", payload.get("prompt"))
        if match:
            payload["hires_fix_denoising_strength"] = match.group(1)
            payload["prompt"] = re.sub(r"<hires_fix:(\d+\.\d+)>", "", payload["prompt"])

        # To change hires_fix_denoising_strength
        #
'''

search_string_dimensions = '                newwidth, newheight = ImageUtils.calculate_source_image_size(width, height)'
new_code_dimensions = r'''                #newwidth, newheight = ImageUtils.calculate_source_image_size(width, height)
                #
                # To change hires-fix initial dimensions
                import re
                # Get the dimensions from the prompt
                match = re.search(r"<width\*height:(\d+\*\d+)>", payload.get("prompt"))
                if match:
                    partial_match = match.group(1)
                    newwidth = partial_match.split('*')[0]
                    newheight = partial_match.split('*')[1]
                    payload["prompt"] = re.sub(r"<width\*height:(\d+\*\d+)>", "", payload["prompt"])
                else:
                    newwidth, newheight = ImageUtils.calculate_source_image_size(width, height)

                # To change hires-fix initial dimensions
                #
'''

# only execute this if experimental is True
if (experimental):
    # read the file
    with open(horde_py_path, 'r') as file:
        lines = file.readlines()

    found_line_number = -1

    # find the target line "search_string"
    for i, line in enumerate(lines):
        if search_string in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code" to the data
    if found_line_number != -1:
        lines.insert(found_line_number - 1, new_code)
    
    found_line_number = -1

    # find the target line "search_string_dimensions"
    for i, line in enumerate(lines):
        if search_string_dimensions in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code_dimensions" to the data
    if found_line_number != -1:
        lines.pop(found_line_number)
        lines.insert(found_line_number, new_code_dimensions)

    # rewrite horde.py
    with open(horde_py_path, 'w') as file:
        file.writelines(lines)


In [ ]:
# @title ## EXPERIMENTAL - Lora/TI injection fix, Lora size limit up (220MB -> 1GB) and login-gated Loras support { display-mode: "form" }

# Modify lora.py as a fix for the worker update of 12/19/2023. Also, increase lora size limit to 1gb and add support for login-gated loras

# To undo the experimental changes, uncomment the next 2 lines (delete the #) and run this cell again, then rerun the worker (the last cell)
#!wget -O /content/regen/lib/python3.10/site-packages/hordelib/model_manager/lora.py https://raw.githubusercontent.com/Haidra-Org/hordelib/v2.2.3/hordelib/model_manager/lora.py
#experimental = False

lora_py_path = '/content/regen/lib/python3.10/site-packages/hordelib/model_manager/lora.py'

search_string = '                                self.delete_oldest_lora()'
new_code = r'''                                # I added this logger to check it and it is indeed what's deleting the lora
                                logger.error(
                                  f"This thinks it's full or whatever",
                                )
                                # I commented this next line to skip the lora delete and it started wokring as I wanted
                                #self.delete_oldest_lora()
'''

search_string_2 = '        if lora["adhoc"] and lora["size_mb"] > 220:'
new_code_2 = f'''        if lora["adhoc"] and lora["size_mb"] > 1024:\n'''


search_string_3 = '                    response = requests.get(lora["url"], timeout=self.REQUEST_DOWNLOAD_TIMEOUT)'
new_code_3 = f'''                    response = requests.get(lora["url"] + "?token=cae554bcc138d97a9323856c2dee1158", timeout=self.REQUEST_DOWNLOAD_TIMEOUT)\n'''



#search_string_2 = '                if lora["versions"][lora_version]["adhoc"] and lora["versions"][lora_version]["size_mb"] > 220:'
#new_code_2 = f'''        if lora["versions"][lora_version]["adhoc"] and lora["versions"][lora_version]["size_mb"] > 1024:\n'''

#search_string_3 = '                    response = requests.get(lora["versions"][version]["url"], timeout=self.REQUEST_DOWNLOAD_TIMEOUT)'
#new_code_3 = f'''                    response = requests.get(lora["versions"][version]["url"] + "?token=cae554bcc138d97a9323856c2dee1158", timeout=self.REQUEST_DOWNLOAD_TIMEOUT)\n'''


# only execute this if experimental is True
if (experimental):
    # read the file
    with open(lora_py_path, 'r') as file:
        lines = file.readlines()

    found_line_number = -1

    ### fix for lora injection 
    # find the target line "search_string"
    for i, line in enumerate(lines):
        if search_string in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code" to the data
    if found_line_number != -1:
        lines.pop(found_line_number)
        lines.insert(found_line_number, new_code)

    ### to increase lora size limit
    # find the target line "search_string_2"
    for i, line in enumerate(lines):
        if search_string_2 in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code_2" to the data
    if found_line_number != -1:
        lines.pop(found_line_number)
        lines.insert(found_line_number, new_code_2)

    ### to allow login-gated loras
    # find the target line "search_string_3"
    for i, line in enumerate(lines):
        if search_string_3 in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code_3" to the data
    if found_line_number != -1:
        lines.pop(found_line_number)
        lines.insert(found_line_number, new_code_3)


    # rewrite lora.py
    with open(lora_py_path, 'w') as file:
        file.writelines(lines)


In [ ]:
# @title ## EXPERIMENTAL - Show popped job and extra info { display-mode: "form" }

# To modify process_manager.py to show popped job and extra info.
# Repeat the above to modify process_manager.py as a fix for the update of 12/19/2023

# To undo the experimental changes, uncomment the next 2 lines (delete the #) and run this cell again, then rerun the worker (the last cell)
#!wget -O /content/horde-worker-reGen/horde_worker_regen/process_management/process_manager.py https://raw.githubusercontent.com/Haidra-Org/horde-worker-reGen/main/horde_worker_regen/process_management/process_manager.py
#experimental = False

process_manager_py_path = '/content/horde-worker-reGen/horde_worker_regen/process_management/process_manager.py'


# only execute this if experimental is True
if (experimental):
    # read the file
    with open(process_manager_py_path, 'r') as file:
        lines = file.readlines()

    found_line_number = -1

    ##
    ## popped job - search_string_1, new_code_1
    search_string_1 = '        logger.info(f"Popped job {job_pop_response.id_} (model: {job_pop_response.model})")'
    new_code_1 = r'''        logger.success(f"Popped job {job_pop_response.id_} (model: {job_pop_response.model})")
'''

    # find the target line "search_string_1"
    for i, line in enumerate(lines):
        if search_string_1 in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code_1" to the data
    if found_line_number != -1:
        lines.pop(found_line_number)
        lines.insert(found_line_number, new_code_1)
    ## popped job
    ##
    found_line_number = -1

    ##
    ## number and ID of loras - search_string_lora, new_code_lora
    search_string_lora = '            if next_job.payload.loras:'
    new_code_lora = r'''            ##
            import re
            matches = ""
            matches = re.findall(r"<lora:(-?\d+:-?\d+\.\d+:-?\d+\.\d+).*?>", next_job.payload.prompt)
            lora_array = []
            # Array containing all loras in the prompt
            for match in matches:
                lora_id = match.split(':')[0]
                lora_array.append(lora_id)

            if next_job.payload.loras or len(lora_array):
                if extra_info:
                    extra_info += ", "
                lora_payload_names = [entry.name for entry in next_job.payload.loras]
                extra_info += f"{len(next_job.payload.loras)} (payload: {lora_payload_names if lora_payload_names else 'None'}) and {len(lora_array)} (injection: {lora_array if lora_array else 'None'}) LoRAs"
            ##
'''

    # find the target line "search_string_lora"
    for i, line in enumerate(lines):
        if search_string_lora in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code_lora" to the data
    if found_line_number != -1:
        lines.pop(found_line_number)
        lines.pop(found_line_number)
        lines.pop(found_line_number)
        lines.pop(found_line_number)
        lines.insert(found_line_number, new_code_lora)
    ## number and ID of loras
    ##
    found_line_number = -1

    ##
    ## number and ID of tis - search_string_ti, new_code_ti
    search_string_ti = '            if next_job.payload.tis:'
    new_code_ti = r'''            ##
            matches = ""
            matches = re.findall(r"embedding:(-?\d+:-?\d+\.\d+)", next_job.payload.prompt)
            ti_array = []
            # Array containing all embeddings in the prompt
            for match in matches:
                ti_id = match.split(':')[0]
                ti_array.append(ti_id)

            if next_job.payload.tis or len(ti_array):
                if extra_info:
                    extra_info += ", "
                ti_payload_names = [entry.name for entry in next_job.payload.tis]
                extra_info += f"{len(next_job.payload.tis)} (payload: {ti_payload_names if ti_payload_names else 'None'}) and {len(ti_array)} (injection: {ti_array if ti_array else 'None'}) TIs"
            ##
'''

    # find the target line "search_string_ti"
    for i, line in enumerate(lines):
        if search_string_ti in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code_ti" to the data
    if found_line_number != -1:
        lines.pop(found_line_number)
        lines.pop(found_line_number)
        lines.pop(found_line_number)
        lines.pop(found_line_number)
        lines.insert(found_line_number, new_code_ti)
    ## number and ID of tis
    ##
    found_line_number = -1

    ##
    ## extra info - search_string_extra, new_code_extra
    search_string_extra = '                logger.info(extra_info)'
    new_code_extra = f'''                logger.success(extra_info)\n'''

    ### to allow login-gated lora
    # find the target line "search_string_extra"
    for i, line in enumerate(lines):
        if search_string_extra in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code_extra" to the data
    if found_line_number != -1:
        lines.pop(found_line_number)
        lines.insert(found_line_number, new_code_extra)
    ## extra info
    ##

    # rewrite process_manager.py
    with open(process_manager_py_path, 'w') as file:
        file.writelines(lines)


In [ ]:
# @title 6.- Run download models { display-mode: "form" }

# Make sure you have the correct path based on any `cd` commands above
!cd /content

!source ../regen/bin/activate;python download_models.py

In [ ]:
# @title 7.- Run the worker { display-mode: "form" }

# Make sure you have the correct path based on any `cd` commands above
!cd /content

if (logs):
    !source ../regen/bin/activate;python run_worker.py -vv
else:
    !source ../regen/bin/activate;python run_worker.py
